# Análise Comparativa de Modelos

<div><h2>Visão Geral</h2></div>

O objetivo deste notebook é realizar a etapa de preparação de dados e modelagem preditiva com o conjunto de dados `Tips`. 

A metodologia adotada foi estruturada nas seguintes etapas:

* **Obtenção de Dados**
* **Padronização de Variáveis**
* **Preparação dos Dados**

Para a realização da análise estatística e visualização dos dados foram utilizadas as seguintes bibliotecas do Python: *Pandas*, *Numpy*, *Matplotlib*, *Seaborn* e *Scikit-learn*. 

---

<div><h2>Metodologia</h2></div>

### Obtenção de Dados:

Nesta etapa, foi configurado o ambiente com a importação das bibliotecas do Python necessárias para preparação e modelagem de dados, como *Pandas*, *Numpy*, *Scikit-Learn* e *Scipy*. Em seguida, foi carregado o conjunto de dados `Tips`, carregado o dicionário de dados e apresentada uma caracterização geral dos dados.

* Base de dados: https://github.com/atlantico-academy/datasets/blob/main/tips.csv
* Informações sobre o conjunto de dados: https://rdrr.io/cran/reshape2/man/tips.html 


### Padronização de Variáveis:

Nesta etapa, realizou-se a padronização das variáveis do conjunto de dados. As variáveis categóricas foram convertidas para o tipo `category`. Além disso, algumas colunas foram criadas para aprofundar a análise e as demais foram renomedas para o padrão `snake_case`.

### Preparação de Dados:

Esta etapa envolve a limpeza e transformação de dados necessárias para garantir que os dados estejam prontos para a modelagem. Isso inclui:

- **Tratamento de dados faltantes:** As colunas com valores faltantes foram tratadas utilizando o **Simple Imputer** do Scikit-Learn. Para as variáveis numéricas, os valores faltantes foram imputados utilizando a mediana. Já para as variáveis categóricas, foi utilizada a imputação de valores faltantes pela moda.

- **Tratamento de outliers**: Os outliers foram tratados utilizando o **Método de Capping** baseado no Intervalo Interquartil (IQR), que limita os valores dos dados dentro de intervalos definidos para evitar distorções. Esse método estabelece limites superior e inferior com base nos percentis especificados (por padrão, 5% e 95%) e ajusta os valores extremos para esses limites.

- **Codificação de variáveis:** As variáveis nominais foram codificadas utilizando o **One-Hot Encoder** do Scikit-Learn, facilitando a aplicação em algoritmos de aprendizado de máquina. As variáveis ordinais foram codificadas com **Ordinal Encoder** do Scikit Learn quando houver uma ordem específica para manter.

- **Normalização de variáveis:** As variáveis numéricas foram identificadas e normalizadas utilizando o **Standard Scaler** do Scikit-Learn, com o objetivo de normalizar as variáveis numéricas em uma escala comum.
    
--- 

<div><h2>Sumário</h2></div>

1. [Obtenção de dados](#obtencao_dados)<br>
1.1. [Configuração do ambiente](#configuracao_ambiente)<br>
1.2. [Leitura do Conjunto de dados](#leitura_dados)<br>
1.3. [Dicionário de dados](#dicionario_dados)<br>
1.4. [Conjunto de dados](#conjunto_dados)<br>
1.5. [Dados faltantes ou nulos](#dados_faltantes)<br>
1.6. [Identificação e visualização de outliers](#outliers)<br>
2. [Padronização de variáveis](#padronizacao)<br>
2.1. [Conversão de variáveis categóricas](#dados_categoricos)<br>
2.2. [Conversão e separação de variáveis temporais](#dados_temporais)<br>
2.3. [Renomeando colunas](#renomear_colunas)<br>
2.4. [Atualização do dicionário de dados](#dicionario_atualizado)<br>
3. [Preparação de dados](#preparacao)<br>
3.1. [Variável alvo e variáveis preditoras](#variavel_alvo)<br>
3.2. [Tratamento de outliers](#tratamento_outliers)<br>
3.3. [Pipeline de pré-processamento de dados](#tratamento_dados)<br>
3.4. [Verificação de dados após o pré-processamento](#verificacao_dados)

---

<div id='descricao'><h2>1. Obtenção de dados</h2></div>

Nessa etapa, serão obtidos novamente os arquivos brutos de dados e o dicionário antes de iniciar o pre-processamento.

<div id='leitura_dados'><h3>1.1. Configurações do ambiente</h3></div> 

In [1]:
# Configuração do Ambiente
from IPython.display import display, Markdown, HTML
import pandas as pd
import numpy as np

from scipy import stats
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin

<div id='leitura_dados'><h3>1.2. Leitura do conjunto de dados</h3></div> 

In [2]:
# Apresentando o conjunto de dados
df = pd.read_csv('../data/raw/data.csv')
display(df.head())

# Exibição da descrição do conjunto de dados
display(Markdown(
    "O conjunto de dados contém informações sobre gorjetas recebidas por um garçom durante um intervalo " 
    "de dois meses e meio na década de 1990, enquanto trabalhava em um restaurante norte-americano. " 
    "Inclui dados sobre o **total da conta**, a **gorjeta**, o **sexo dos clientes**, o **status de fumante**, "
    "o **dia da semana** e o **período do dia** em que foi realizado o consumo, e a **quantidade de pessoas na mesa**.\n\n"
    "---"
))


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


O conjunto de dados contém informações sobre gorjetas recebidas por um garçom durante um intervalo de dois meses e meio na década de 1990, enquanto trabalhava em um restaurante norte-americano. Inclui dados sobre o **total da conta**, a **gorjeta**, o **sexo dos clientes**, o **status de fumante**, o **dia da semana** e o **período do dia** em que foi realizado o consumo, e a **quantidade de pessoas na mesa**.

---

<div id='dicionario_dados'><h3>1.3. Dicionário de dados</h3></div> 

In [3]:
#Dicionário de dados
df_dict = pd.read_csv('../data/external/dictionary.csv')
display(HTML(df_dict.to_html(index=False)))
display(Markdown(
    "---\n"
    "O dicionário de dados possui **7 variáveis**, sendo elas:\n" 
    "* **2** quatitativas contínuas\n"
    "* **1** quantitativa discreta\n" 
    "* **3** qualitativas nominais\n"
    "* **1** qualitativa ordinal"
    "\n\n"
    "---"
))

variavel,descricao,tipo,subtipo
total_bill,Total da conta em dólares,quantitativa,contínua
tip,Gorjeta recebida em dólares,quantitativa,contínua
sex,Gênero dos clientes,qualitativa,nominal
smoker,Presença de fumantes na mesa,qualitativa,nominal
day,Dia da semana em que a gorjeta foi recebida,qualitativa,ordinal
time,Período em que as refeições ocorreram,qualitativa,nominal
size,Quantidade de pessoas na mesa,quantitativa,discreta


---
O dicionário de dados possui **7 variáveis**, sendo elas:
* **2** quatitativas contínuas
* **1** quantitativa discreta
* **3** qualitativas nominais
* **1** qualitativa ordinal

---

<div id='conjunto_dados'><h3>1.4. Conjunto de dados</h3></div> 

In [4]:
# Resumo das informações do conjunto de dados
display(Markdown("*Informações Gerais do Dataframe:* \n\n"))
df.info()
print("\n")

# Visualizando a estatística descritiva do conjunto de dados
display(Markdown("*Estatística Descritiva* \n\n"))
display(df.describe(include='all'))

*Informações Gerais do Dataframe:* 



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  244 non-null    float64
 1   tip         244 non-null    float64
 2   sex         244 non-null    object 
 3   smoker      244 non-null    object 
 4   day         244 non-null    object 
 5   time        244 non-null    object 
 6   size        244 non-null    int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 13.5+ KB




*Estatística Descritiva* 



,total_bill,tip,sex,smoker,day,time,size
count,244.000000,244.000000,244,244,244,244,244.000000
unique,NaN,NaN,2,2,4,2,NaN
top,NaN,NaN,Male,No,Sat,Dinner,NaN
freq,NaN,NaN,157,151,87,176,NaN
mean,19.785943,2.998279,NaN,NaN,NaN,NaN,2.569672
std,8.902412,1.383638,NaN,NaN,NaN,NaN,0.951100
min,3.070000,1.000000,NaN,NaN,NaN,NaN,1.000000
25%,13.347500,2.000000,NaN,NaN,NaN,NaN,2.000000
50%,17.795000,2.900000,NaN,NaN,NaN,NaN,2.000000
75%,24.127500,3.562500,NaN,NaN,NaN,NaN,3.000000


* O conjunto de dados possui **244 linhas** e **7 colunas**.

* Há 1 variável do tipo **integer**.

* Há 2 variáveis do tipo **float**.

* As demais variáveis são classificadas como **object**, porém são do tipo **string**.

---

<div id='dados_faltantes'><h3>1.5. Dados faltantes ou nulos</h3></div> 

In [5]:
# Exibindo a soma de valores ausentes por coluna
missing_values_by_column = df.isnull().sum().loc[lambda x: x > 0]
missing_values = df.isnull().any(axis=1).sum()

if missing_values == 0:
    print("O conjunto não possui dados faltantes.") 
else: 
    print(f"O conjunto possui {missing_values} linhas com dados faltantes.")

display(missing_values_by_column)

O conjunto não possui dados faltantes.


Series([], dtype: int64)

* O conjunto `Tips` não possui dados faltantes ou nulos.

<div id='outliers'><h3>1.6. Identificação de outliers</h3></div> 

In [6]:
# Identificação de outliers usando o método do Intervalo Interquantil (IQR)
numeric_df = df.select_dtypes(include=[np.number])
factor = 1.5

def detect_outliers_iqr(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - factor * IQR
    upper_bound = Q3 + factor * IQR
    return (data < lower_bound) | (data > upper_bound)

# Aplicando a função a todas as colunas numéricas
outliers = numeric_df.apply(detect_outliers_iqr)

# Filtrando os dados para mostrar apenas os outliers
outliers_summary = outliers.sum().sort_values(ascending=False)
outliers_summary = outliers_summary[outliers_summary > 0]
outliers_data = numeric_df.loc[:, outliers_summary.index]
display(Markdown("*Número de Outliers por Coluna:* \n"))
display(outliers_summary)
print("\n")
display(outliers_data.head())

*Número de Outliers por Coluna:* 


total_bill    9
tip           9
size          9
dtype: int64

,total_bill,tip,size
0,16.99,1.01,2
1,10.34,1.66,3
2,21.01,3.50,3
3,23.68,3.31,2
4,24.59,3.61,4


* O cálculo do IQR revelou a presença de **27** outliers distribuídos entre as variáveis `total_bill`, `tip` e `size`, num total de **9** valores cada.

---

<div id='padronizacao'><h2>2. Padronização de variáveis</h2></div>

<div id='dados_categoricos'><h3>2.1. Conversão de variáveis categóricas</h3></div> 

In [7]:
# Conversão De Variaveis Categóricas
nominal_columns = df_dict.query("subtipo == 'nominal'").variavel.tolist()

df[nominal_columns] = df[nominal_columns].astype('category')

display(Markdown("*Tipos dos dados após conversão:* \n\n"))
print(df.dtypes)

*Tipos dos dados após conversão:* 



total_bill     float64
tip            float64
sex           category
smoker        category
day             object
time          category
size             int64
dtype: object


* Conversão das colunas `sex`, `smoker` e `time` do tipo `object` para `category`, sendo este mais eficiente para variáveis categóricas com um número limitado de valores distintos. 

* Após a conversão, são apresentados os tipos dos dados das variáveis atualizado.

---

<div id='ordem_dados'><h3>2.2. Ordenando a variável `day`</h3></div> 

In [8]:
# Definindo a ordem dos dias da semana
days_order = ['Thur', 'Fri', 'Sat', 'Sun']

# Convertendo a coluna 'day' para categórico com a ordem especificada
df['day'] = pd.Categorical(df['day'], categories=days_order, ordered=True)

display(df['day'].unique())

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur' < 'Fri' < 'Sat' < 'Sun']

* Ordenando a variável `day` pela ordem dos dias da semana, assim como convertendo para o tipo categórico.

<div id='novas_colunas'><h3>2.3. Criação de novas variáveis</h3></div> 

In [9]:
# Criando a variável percentual de gorjeta (tip_percentage) em valor percentual e arredondado
df['tip_percentage'] = (df['tip'] / df['total_bill'].replace(0, np.nan) * 100).round(2)

# Criando a variável custo médio por pessoa (cost_per_person)
df['cost_per_person'] = ((df['total_bill'] + df['tip']) / df['size'].replace(0, np.nan)).round(2)

display(df.head())

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,cost_per_person
0,16.99,1.01,Female,No,Sun,Dinner,2,5.94,9.00
1,10.34,1.66,Male,No,Sun,Dinner,3,16.05,4.00
2,21.01,3.50,Male,No,Sun,Dinner,3,16.66,8.17
3,23.68,3.31,Male,No,Sun,Dinner,2,13.98,13.50
4,24.59,3.61,Female,No,Sun,Dinner,4,14.68,7.05


* Criação de novas variáveis para enriquecer os dados e possibilitar mais parâmetros de análise. Foram criadas as seguintes variáveis: `tip_percentage` e `cost_per_person`.

* A variável `tip_percentage` trata-se da relação entre a gorjeta recebida (tip) e o valor total da conta (total_bill). É do tipo quantitativa contínua.

* A variável `cost_per_person` trata do valor médio por cliente que frequente o restaurante, sendo a soma dos valores de gorjeta e total da conta pela quantidade de pessoas na mesa. É do tipo quantitativa contínua.

<div id='dicionario_atualizado'><h3>2.4. Atualização do dicionário de dados</h3></div> 

In [10]:
# Criando as novas entradas para o dicionário de dados
new_columns = pd.DataFrame({
    'variavel': ['tip_percentage', 'cost_per_person'],
    'descricao': [
        'Calcula a porcentagem que a gorjeta representa em relação ao valor total da conta',
        'Calcula o custo médio por pessoa, incluindo a gorjeta',
    ],
    'tipo': ['quantitativa', 'quantitativa'],
    'subtipo': ['contínua', 'contínua']
})

df_dict = pd.concat([df_dict, new_columns], ignore_index=True)

display(HTML(df_dict.to_html(index=False)))

display(Markdown(
    "---\n"
    "O dicionário de dados atualizado possui **11 variáveis**, sendo elas:\n" 
    "* **4** quatitativas contínuas\n"
    "* **1** quantitativa discreta\n" 
    "* **2** qualitativas nominais\n"
    "* **1** qualitativas ordinal"
    "\n\n"
    "---"
))
print(df.dtypes)

variavel,descricao,tipo,subtipo
total_bill,Total da conta em dólares,quantitativa,contínua
tip,Gorjeta recebida em dólares,quantitativa,contínua
sex,Gênero dos clientes,qualitativa,nominal
smoker,Presença de fumantes na mesa,qualitativa,nominal
day,Dia da semana em que a gorjeta foi recebida,qualitativa,ordinal
time,Período em que as refeições ocorreram,qualitativa,nominal
size,Quantidade de pessoas na mesa,quantitativa,discreta
tip_percentage,Calcula a porcentagem que a gorjeta representa em relação ao valor total da conta,quantitativa,contínua
cost_per_person,"Calcula o custo médio por pessoa, incluindo a gorjeta",quantitativa,contínua


---
O dicionário de dados atualizado possui **11 variáveis**, sendo elas:
* **4** quatitativas contínuas
* **1** quantitativa discreta
* **2** qualitativas nominais
* **1** qualitativas ordinal

---

total_bill          float64
tip                 float64
sex                category
smoker             category
day                category
time               category
size                  int64
tip_percentage      float64
cost_per_person     float64
dtype: object


<div id='preparacao'><h2>3. Preparação de Dados</h2></div>

Nesta seção, serão realizadas a preparação de dados com a separação da variáveis alvo e variáveis preditoras, o tratamento de dados faltantes e discrepantes, a normalização e a codificação em um pipeline de pré-processamento de dados.

<div id='variavel_alvo'><h3>3.1. Variável alvo e preditoras</h3></div> 

In [11]:
#Variável alvo
target_column = 'tip'

#Variáveis Nominais
nominal_columns = (
    df_dict
    .query("subtipo == 'nominal' and variavel != @target_column")
    .variavel
    .to_list()
)

#Variáveis Ordinais
ordinal_columns = (
    df_dict
    .query("subtipo == 'ordinal' and variavel != @target_column")
    .variavel
    .to_list()
)

#Variáveis Discretas
discrete_columns = (
    df_dict
    .query("subtipo == 'discreta' and variavel != @target_column")
    .variavel
    .to_list()
)

#Variáveis Contínuas
continuous_columns = (
    df_dict
    .query("subtipo == 'contínua' and variavel != @target_column")
    .variavel
    .to_list()
)

# Isolando a variável alvo das variáveis preditoras
X = df.drop(columns=[target_column], axis=1)
y = df[target_column]
display(X)

display(Markdown(
    f"- Variável alvo: `{target_column}` <br>\n"
    f"- Variáveis qualitativas nominais: `{nominal_columns}` \n"    
    f"- Variáveis qualitativas ordinais: `{ordinal_columns}` \n"
    f"- Variáveis quantitativas discretas: `{discrete_columns}` \n" 
    f"- Variáveis quantitativas contínuas: `{continuous_columns}`\n"   
    "---"
))

,total_bill,sex,smoker,day,time,size,tip_percentage,cost_per_person
0,16.99,Female,No,Sun,Dinner,2,5.94,9.00
1,10.34,Male,No,Sun,Dinner,3,16.05,4.00
2,21.01,Male,No,Sun,Dinner,3,16.66,8.17
3,23.68,Male,No,Sun,Dinner,2,13.98,13.50
4,24.59,Female,No,Sun,Dinner,4,14.68,7.05
...,...,...,...,...,...,...,...,...
239,29.03,Male,No,Sat,Dinner,3,20.39,11.65
240,27.18,Female,Yes,Sat,Dinner,2,7.36,14.59
241,22.67,Male,Yes,Sat,Dinner,2,8.82,12.34
242,17.82,Male,No,Sat,Dinner,2,9.82,9.78


- Variável alvo: `tip` <br>
- Variáveis qualitativas nominais: `['sex', 'smoker', 'time']` 
- Variáveis qualitativas ordinais: `['day']` 
- Variáveis quantitativas discretas: `['size']` 
- Variáveis quantitativas contínuas: `['total_bill', 'tip_percentage', 'cost_per_person']`
---

<div id='tratamento_outliers'><h3>3.2. Tratamento de dados discrepantes</h3></div> 

In [12]:
# Tratamento de outliers pelo Método de Capping

# Classe para aplicar o Capping
class CappingOutliers(BaseEstimator, TransformerMixin):
    def __init__(self, lower_percentile=5, upper_percentile=95):
        self.lower_percentile = lower_percentile
        self.upper_percentile = upper_percentile

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_capped = X.copy()
        for col in range(X_capped.shape[1]):
            lower_bound = np.percentile(X_capped[:, col], self.lower_percentile)
            upper_bound = np.percentile(X_capped[:, col], self.upper_percentile)
            X_capped[:, col] = np.clip(X_capped[:, col], lower_bound, upper_bound)
        return X_capped

* Para o tratamento de outliers, foi utilizado o **Método de Capping** (ou Winsorizing). Esta técnica limita os valores extremos dos dados a intervalos definidos por percentis, tipicamente entre 5% e 95%. 

* Valores abaixo do limite inferior de 5% são substituídos por esse limite inferior, enquanto valores acima do limite superior de 95% são ajustados para o limite superior. Essa abordagem reduz a influência dos outliers nos modelos de machine learning e minimiza as distorções nos dados.

<div id='tratamento_dados'><h3>3.3. Tratamento de dados faltantes, normalização e codificação</h3></div> 

In [13]:
# Tratamento de dados faltantes e discrepantes, codificação e normalização

# Pipeline para variáveis ordinais
ordinal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), # Tratamento de dados faltantes
    ('encoding', OrdinalEncoder(categories=[days_order])), # Codificação de variáveis
])

# Pipeline para variáveis nominais
nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), # Tratamento de dados faltantes
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')), # Codificação de variáveis
])

# Pipeline para variáveis discretas
discrete_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='median')), # Tratamento de dados faltantes
    ('capping', CappingOutliers()), # Tratamento de outliers
    ('normalization', StandardScaler()) # Normalização de dados 
])

# Pipeline para variáveis contínuas
continuous_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='median')), # Tratamento de dados faltantes
    ('capping', CappingOutliers()), # Tratamento de outliers
    ('normalization', StandardScaler()) # Normalização de dados 
])

# Combinação dos pipelines em um ColumnTransformer
preprocessor = ColumnTransformer([
    ('ordinal', ordinal_preprocessor, ordinal_columns),
    ('nominal', nominal_preprocessor, nominal_columns),
    ('discrete', discrete_preprocessor, discrete_columns),
    ('continuous', continuous_preprocessor, continuous_columns),
])

# Aplicando o pipeline de pré-processamento aos dados X
X_transformed = preprocessor.fit_transform(X)

In [14]:
# Dimensão dos dados X após o pré-processamento
X_transformed.shape

(244, 8)

In [15]:
# Apresentando o pré-processamento dos dados X
display(X_transformed)

array([[ 3.        ,  0.        ,  0.        , ..., -0.32825704,
        -1.82220768, -0.00609118],
       [ 3.        ,  1.        ,  0.        , ..., -1.16523781,
         0.06743346, -1.36394465],
       [ 3.        ,  1.        ,  0.        , ...,  0.17770728,
         0.20825333, -0.29886125],
       ...,
       [ 2.        ,  1.        ,  1.        , ...,  0.38663781,
        -1.60162835,  1.17204379],
       [ 2.        ,  1.        ,  0.        , ..., -0.22379177,
        -1.37077609,  0.26904213],
       [ 0.        ,  0.        ,  0.        , ..., -0.10296447,
         0.04896528,  0.66057801]])

* **Pipeline de variáveis nominais:** Inclui a imputação de valores faltantes pela moda, codificação das variáveis com **One-Hot Encoder**, onde o método de **Dummy Encoding** é utilizado, excluindo a primeira categoria para evitar multicolinearidade. 

* **Pipeline de variáveis ordinais:** Inclui a imputação de valores faltantes pela moda e codificação das variáveis com **Ordinal Encoder**.

* **Pipeline de variáveis contínuas e discretas:** Inclui a imputação de valores faltantes pela mediana, o tratamento de outliers pelo **Método de Capping** e a normalização das variáveis com o **Standard Scaler**. 

* **Combinação dos pipelines:** Ambos os pipelines foram combinados em um **ColumnTransformer**, que aplica o pré-processamento adequado a cada grupo de colunas pré-estabelecido.

<div id='verificacao_dados'><h3>3.4. Verificação de dados após o pré-processamento</h3></div> 

#### Verificação de dados faltantes residuais

In [17]:
# Verificando se ainda existem dados faltantes ou nulos após a etapa de pré-processamento
display(Markdown("*Número de dados faltantes após o pré-processamento:* \n"))
np.isnan(X_transformed).sum()

*Número de dados faltantes após o pré-processamento:* 


0

#### Verificação de dados discrepantes residuais

In [18]:
# Verificando se ainda existem dados discrepantes após a etapa de pré-processamento
def detect_outliers(data, column_names, threshold=3):
    outliers_info = {}
    z_scores = np.abs(stats.zscore(data))
    for i, column_name in enumerate(column_names):
        outliers = (z_scores[:, i] > threshold)
        outliers_info[column_name] = np.sum(outliers)
    return outliers_info

# Lista dos nomes das colunas originais, incluindo as colunas contínuas e discretas
column_names = continuous_columns + discrete_columns

# Aplicando a detecção de outliers com Z-Score Ajustado
outliers = detect_outliers(X_transformed, column_names)

# Exibindo o número de outliers por coluna
display(Markdown("*Número de outliers após o pré-processamento:*\n"))
for column, count in outliers.items():
    print(f"{column}: {count}")

*Número de outliers após o pré-processamento:*


total_bill: 0
tip_percentage: 0
cost_per_person: 0
size: 0


* Após a transformação, foi confirmada a ausência de dados faltantes, com a soma de valores `NaN` resultando em `0`.

* Por meio do método Z-score, foi observada também a ausência de valores discrepantes após a etapa de pré-processamento.

---